In [1]:
pip install transformers datasets evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 227.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 118.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 170.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.6/786.6 kB 130.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 172.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

ds = load_dataset("facebook/multilingual_librispeech", "german")

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/3469 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/469942 [00:00<?, ? examples/s]

Generating 9_hours split:   0%|          | 0/2194 [00:00<?, ? examples/s]

Generating 1_hours split:   0%|          | 0/241 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/62 [00:00<?, ?it/s]

In [14]:
# Install the necessary packages
!pip install soundfile>=0.12.1
!pip install librosa 
!apt-get install -y libsndfile1



import soundfile as sf
from datasets import load_dataset

# Load the dataset
mls = load_dataset("facebook/multilingual_librispeech", "german", split="train", streaming=True)

# Print the first example from the dataset
print(next(iter(mls)))

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libsndfile1


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

RuntimeError: Decoding 'opus' files requires system library 'libsndfile'>=1.0.31, You can try to update `soundfile` python library: `pip install "soundfile>=0.12.1"`. 

In [15]:
# Split the dataset into train and test sets
split_ds = ds['train'].train_test_split(test_size=0.2)

In [16]:
# Removing columns that are not needed
split_ds = split_ds.remove_columns(["speaker_id", "chapter_id", "id"])

In [24]:
# Display the first example in the training set
print(split_ds['train'])

Dataset({
    features: ['audio', 'original_path', 'begin_time', 'end_time', 'transcript', 'audio_duration', 'file'],
    num_rows: 375953
})


In [19]:
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

# Load the Whisper processor and model
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3")

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [28]:
# Install the necessary packages
!pip install librosa soundfile
!apt-get install -y libsndfile1

import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from datasets import load_dataset, Audio

# Load the dataset with streaming=True
mls = load_dataset("facebook/multilingual_librispeech", "german", split="train", streaming=True)

# Load the Whisper model and processor
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3")

# Access a single example from the dataset
example = next(iter(mls))

# Extract the audio array and sampling rate from the example
audio = example['audio']['array']
sampling_rate = example['audio']['sampling_rate']

# Process the audio signal
inputs = processor(audio, sampling_rate=sampling_rate, return_tensors="pt")

# Generate transcription using the Whisper model
with torch.no_grad():
    logits = model(**inputs).logits

# Decode the output logits to generate the transcription
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print(transcription)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libsndfile1


Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

RuntimeError: Decoding 'opus' files requires system library 'libsndfile'>=1.0.31, You can try to update `soundfile` python library: `pip install "soundfile>=0.12.1"`. 

In [30]:
from datasets import load_dataset, Audio

# Load the German portion of the Multilingual LibriSpeech dataset
ds = load_dataset("facebook/multilingual_librispeech", "german")

# Resample the audio to 16,000 Hz, which is required by the Whisper model
ds = ds.cast_column("audio", Audio(sampling_rate=16000))

# Verify the first sample in the training set
print(ds['train'])

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/62 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'original_path', 'begin_time', 'end_time', 'transcript', 'audio_duration', 'speaker_id', 'chapter_id', 'file', 'id'],
    num_rows: 469942
})


In [31]:
mls = load_dataset("facebook/multilingual_librispeech", "german", split="train", streaming=True)

# Define a generator function to apply the uppercase transformation
def uppercase_streamed_data(dataset):
    for example in dataset:
        example['text'] = example['text'].upper()
        yield example

# Apply the function using the generator
uppercased_mls = uppercase_streamed_data(mls)

# Print the first example after applying the uppercase transformation
print(next(iter(uppercased_mls)))

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/63 [00:00<?, ?it/s]

RuntimeError: Decoding 'opus' files requires system library 'libsndfile'>=1.0.31, You can try to update `soundfile` python library: `pip install "soundfile>=0.12.1"`. 

In [33]:
 #Define the preprocessing function
def prepare_dataset(batch):
    audio = batch["audio"]
    # Process the audio and text using the Whisper processor
    batch = processor(audio["array"], sampling_rate=audio["sampling_rate"], text=batch["text"])
    # Add input_length for later use (e.g., for batching)
    batch["input_length"] = len(batch["input_values"][0])
    return batch

# Apply the preprocessing function to the dataset
encoded_ds = ds.map(prepare_dataset, remove_columns=ds.column_names["train"], num_proc=4)

# Verify the first processed example
print(encoded_ds['train'])

Map (num_proc=4):   0%|          | 0/3469 [00:00<?, ? examples/s]

RuntimeError: Decoding 'opus' files requires system library 'libsndfile'>=1.0.31, You can try to update `soundfile` python library: `pip install "soundfile>=0.12.1"`. 

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import AutoProcessor

# Initialize the Whisper processor
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")

@dataclass
class DataCollatorCTCWithPadding:
    processor: AutoProcessor
    padding: Union[bool, str] = "longest"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they have to be of different lengths and need different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        # Pad input features
        batch = self.processor.pad(input_features, padding=self.padding, return_tensors="pt")

        # Pad label features
        labels_batch = self.processor.pad(label_features, padding=self.padding, return_tensors="pt")

        # Replace padding with -100 to ignore loss correctly in the CTC loss function
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # Combine input values and labels into a single dictionary
        batch["labels"] = labels

        return batch

# Instantiate  DataCollatorCTCWithPadding
data_collator = DataCollatorCTCWithPadding(processor=processor, padding="longest")


In [37]:
import evaluate
import numpy as np
from transformers import AutoProcessor

# Initialize the Whisper processor
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")

# Load the WER metric
wer = evaluate.load("wer")

# Define the compute_metrics function
def compute_metrics(pred):
    # Get the model's logits (predictions)
    pred_logits = pred.predictions

    # Convert logits to predicted IDs (using argmax to get the highest probability token at each position)
    pred_ids = np.argmax(pred_logits, axis=-1)

    # Replace label padding tokens (-100) with the tokenizer's pad token ID
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    # Decode the predicted IDs into text
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)

    # Decode the labels (reference text)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False, skip_special_tokens=True)

    # Compute the WER by comparing predictions with references
    wer_score = wer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer_score}

In [42]:
!pip install accelerate -U
!pip install transformers[torch] -U

from transformers import AutoModelForSpeechSeq2Seq, TrainingArguments, Trainer, AutoProcessor

# Load the Whisper model and processor
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3")
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./whisper-asr",  # where the model checkpoints will be saved
    per_device_train_batch_size=8,  # batch size per device during training
    gradient_accumulation_steps=2,  # accumulate gradients before updating model weights
    learning_rate=1e-5,  # learning rate for optimizer
    warmup_steps=500,  # warmup steps for learning rate scheduler
    max_steps=2000,  # total number of training steps
    gradient_checkpointing=True,  # save memory by using gradient checkpointing
    fp16=True,  # use mixed precision training
    group_by_length=True,  # group sequences of similar lengths together in batches
    eval_strategy="steps",  # evaluate the model every few steps
    per_device_eval_batch_size=8,  # batch size per device during evaluation
    save_steps=1000,  # save checkpoint every 1000 steps
    eval_steps=1000,  # evaluate every 1000 steps
    logging_steps=25,  # log training metrics every 25 steps
    load_best_model_at_end=True,  # load the best model at the end of training
    metric_for_best_model="wer",  # metric to select the best model
    greater_is_better=False,  # lower WER is better
    push_to_hub=True,  # push the model to Hugging Face Hub
)

# Initialize Trainer with the model, training arguments, datasets, and collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_ds['train'],  # your preprocessed training dataset
    eval_dataset=encoded_ds['validation'],  # your preprocessed validation dataset
    tokenizer=processor.feature_extractor,  # use the processor's feature extractor for tokenization
    data_collator=data_collator,  # your custom data collator
    compute_metrics=compute_metrics,  # function to compute WER during evaluation
)

# Start training the model
trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [44]:
from transformers import AutoModelForSpeechSeq2Seq, TrainingArguments, Trainer, AutoProcessor

# Load the Whisper model and processor
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3")
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")

# Define your training arguments
training_args = TrainingArguments(
    output_dir="./whisper-asr",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_ds['train'],
    eval_dataset=encoded_ds['validation'],
    tokenizer=processor.feature_extractor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start training the model
trainer.train()
trainer.push_to_hub()


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [45]:
from datasets import load_dataset, Audio
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq, pipeline

# Load the fine-tuned Whisper model and processor
model = AutoModelForSpeechSeq2Seq.from_pretrained("./whisper-asr")  # path to your fine-tuned model
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")

# Load the German portion of the Multilingual LibriSpeech dataset
dataset = load_dataset("facebook/multilingual_librispeech", "german", split="test")  # Use 'test' split for inference

# Resample the audio to 16kHz if necessary
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Extract the sampling rate and audio file path
sampling_rate = dataset.features["audio"].sampling_rate
audio_file = dataset[0]["audio"]["path"]

# Initialize the ASR pipeline with the fine-tuned model
transcriber = pipeline("automatic-speech-recognition", model=model, tokenizer=processor.tokenizer, feature_extractor=processor.feature_extractor)

# Perform inference on the audio file
transcription = transcriber(audio_file)

# Print the transcription result
print(transcription)


OSError: Incorrect path_or_model_id: './whisper-asr'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [46]:
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

# Load the fine-tuned Whisper model and processor
processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = AutoModelForSpeechSeq2Seq.from_pretrained("./whisper-asr")  # path to your fine-tuned model

# Load the German portion of the Multilingual LibriSpeech dataset
from datasets import load_dataset, Audio

dataset = load_dataset("facebook/multilingual_librispeech", "german", split="test")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# Get the audio data and prepare it for the model
audio_array = dataset[0]["audio"]["array"]
sampling_rate = dataset[0]["audio"]["sampling_rate"]
inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt")

# Pass inputs to the model to get logits
with torch.no_grad():
    logits = model(**inputs).logits

# Get the predicted token IDs with the highest probability
predicted_ids = torch.argmax(logits, dim=-1)

# Decode the predicted token IDs to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

# Output the transcription
print(transcription)

OSError: Incorrect path_or_model_id: './whisper-asr'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [47]:
model.save_pretrained("whisper-large-v3-german-asr")
processor.save_pretrained("whisper-large-v3-german-asr")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}


[]

In [48]:
model.push_to_hub("whisper-large-v3-german-asr")
processor.push_to_hub("whisper-large-v3-german-asr")

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-66cda6ee-408eda0437f283f122ec939f;226f1a6a-e658-475b-8db6-949bec2ec2cb)

Invalid username or password.